In [1]:
from Explainer import *
import utils
import shutil

C:\Users\home\Anaconda3\envs\DL2\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.decomposition.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)


Explainer for torchvision pretrained modesl. ResNet50 is used here.

In [2]:

import torchvision.models as models
import random

m = models.resnet50(pretrained=True)
m = m.cuda()
m.eval()
model = PytorchModelWrapper(m,batch_size=16)


load classes, use path to define dataset, label_path for class name file

In [3]:

from torchvision import transforms
transform = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
classes = ImageNetClassesLoader(path = '/dataset/ILSVRC2012',label_path = "synset_words.txt",preprocess_input=transform)


Target classes are No243 Bull mastiff and No282Tiger cat .

Target layer is the last layer of "layer4" in ResNet50. This is the last layer before GolbalAveragePooling

In [4]:

# set target classes
target_class = [243,282]

title = "dogcat"
try:
    shutil.rmtree('Explainers/'+title)
except:
    pass

# target layer name
layer_name = "layer4"
# set predict layer and target classes
model.target = ("fc",np.array(target_class))
classes.set_target(target_class)


In [5]:

print ("-------------------------------------------------------------------------------------------")

# create an Explainer
Exp = Explainer(title = title,
                layer_name = layer_name,
                classesNos = target_class,
                utils = utils.utils(mode = "torch"),
                n_components = 15,
                reducer_type = "NMF"
               )

# train reducer based on target classes
try:
    Exp.train_model(model,classes)
except:
    Exp.reducer = None
    Exp.train_model(model,classes) 
# generate features 
Exp.generate_features(model,classes)
# save features with visualizations
Exp.save_features()
# generate global explanations
Exp.generate_image_LR_file(classes)
# save the explainer, use load to load it with the same title
Exp.save()

# generate local explanation for catdog.png
from PIL import Image
img_path = "catdog.png"
img = Image.open(img_path).convert("RGB")
x = transforms.ToTensor()(transforms.Resize(classes.target_size)(img))
x = classes.preprocess_input(x).numpy()
x = np.transpose(x,(1,2,0)) #to channel_last
Exp.local_explanations(x,model,classes)

0it [00:00, ?it/s]

-------------------------------------------------------------------------------------------
Training reducer:
Loading data
totally 2 classes


2it [00:27, 13.83s/it]


Dataset too big, train with 0.38 instances
Loading complete, with size of (996, 7, 7, 2048)
Training will take around a minute, please wait for a while...
reducer trained, spent 29.25205135345459 s
fidelity: [0.04590085 0.05337164]


0it [00:00, ?it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [04:03<00:00, 16.23s/it]
0it [00:00, ?it/s]

generating features:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
loading training data
totally 2 classes


2it [00:32, 16.36s/it]
100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  5.05it/s]
0it [00:00, ?it/s]

Generate explanations with fullset condition


2it [00:01,  1.66it/s]


In [6]:
from IPython.display import HTML, display
display(HTML("".join(["<table><tr>",
                      "<td><img src='Explainers/dogcat/GE/243.jpg'></td>",
                      "<td><img src='Explainers/dogcat/GE/282.jpg'></td>",
                      "</tr></table>"])))

,


In [8]:
from IPython.display import HTML, display
display(HTML("".join(["<table><tr>",
                      "<td><img src='Explainers/dogcat/explanations/all/0_243.jpg'></td>",
                      "<td><img src='Explainers/dogcat/explanations/all/0_282.jpg'></td>",
                      "</tr></table>"])))

,
